In [1]:
print("Importing libraries ... ")
import numpy as np
import matplotlib.pyplot as plt
import os
import json
import Puissance_4_bitboard
print("Done")

Importing libraries ... 
Done


In [2]:
file_dir = os.getcwd()+"\\"
file_data = "data_set_Puissance_4.json"

In [3]:
def load_json(json_file):
    with open(json_file) as f:
        data = json.load(f)
        return data

In [4]:
data = load_json(file_dir+file_data)
(X_train, Y_train),\
(X_val, Y_val),\
(X_test, Y_test) = (data["train"]["input"],data["train"]["output"]),\
                   (data["validation"]["input"],data["validation"]["output"]),\
                   (data["test"]["input"],data["test"]["output"])

In [5]:
print("Data in training set : {}".format(len(X_train)))
print("Data in validation set : {}".format(len(X_val)))
print("Data in test set : {}".format(len(X_test)))

Data in training set : 5189933
Data in validation set : 864988
Data in test set : 864990


In [6]:
import tensorflow.keras
from tensorflow.keras import models, layers
from tensorflow.keras import optimizers
from sklearn.neural_network import MLPRegressor
import tensorflow.keras.optimizers as optimizers
import optuna
import pickle

In [7]:
def r2_score(y_true, y_pred):
    return 1 - (np.sum((y_true - y_pred)**2) / np.sum((y_true - y_true.mean())**2))

def nrmse(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    return np.sqrt((np.sum(y_true - y_pred)**2) / len(y_true))

In [8]:
def my_metric(model, input, output):
    predicted_output = model.predict(input)
    return rmse(predicted_output, output)

In [9]:
print("Creating the objective ... ")
class Objective(object):
    def __init__(self, X_train, Y_train, X_val, Y_val,model_save_path):
        # Hold this implementation specific arguments as the fields of the class.
        self._X_train = X_train
        self._Y_train = Y_train
        self._X_val = X_val
        self._Y_val = Y_val
        self._model_path = model_save_path

    def __call__(self, trial):
        # Calculate an objective value by using the extra arguments.
        n_layers = trial.suggest_int("n_layers", 1, 3)
        layers = []
        for i in range(n_layers):
            layers.append(trial.suggest_int(str(i), 4, 128))
        hidden_size = tuple(layers)
        alp  = trial.suggest_float("regularization_term", 1e-5, 1e0, log=True)
        act = trial.suggest_categorical('activation_function', ['identity', 'logistic', 'tanh', 'relu'])
        solv = trial.suggest_categorical('solver', ['sgd', 'adam'])#,'lbfgs'
        if(solv == 'lbfgs'):
            m_iter = trial.suggest_int('max_iteration', 1e6, 1e9, log=True)
            regr = MLPRegressor(random_state=1,\
                            solver = solv,\
                            hidden_layer_sizes=hidden_size,\
                            activation = act,\
                            max_iter=m_iter,\
                            max_fun = 1000,\
                            alpha=alp).fit(self._X_train, self._Y_train)

        elif(solv == 'sgd'):
            m_iter = trial.suggest_int('max_iteration', 1e2, 1e6, log=True)
            b_size = trial.suggest_int("batch_size", 2**5, 2**12, log=True)
            mom = trial.suggest_float("momentum", 0, 1)
            learn_r_init = trial.suggest_float("learning_rate_init", 1e-4, 1e0, log=True)
            learn_r = trial.suggest_categorical('learning_rate', ['constant', 'invscaling', 'adaptive'])    
            regr = MLPRegressor(random_state=1,\
                            solver = solv,\
                            hidden_layer_sizes=hidden_size,\
                            activation = act,\
                            batch_size=b_size,\
                            momentum=mom,\
                            max_iter=m_iter,\
                            alpha=alp,\
                            learning_rate=learn_r,\
                            learning_rate_init=learn_r_init).fit(self._X_train, self._Y_train)

        else:
            m_iter = trial.suggest_int('max_iteration', 1e2, 1e6, log=True)
            b_size = trial.suggest_int("batch_size", 2**5, 2**12, log=True)
            learn_r_init = trial.suggest_float("learning_rate_init", 1e-4, 1e0, log=True)   
            regr = MLPRegressor(random_state=1,\
                            solver = solv,\
                            hidden_layer_sizes=hidden_size,\
                            activation = act,\
                            batch_size=b_size,\
                            max_iter=m_iter,\
                            alpha=alp,\
                            learning_rate_init=learn_r_init).fit(self._X_train, self._Y_train)

        with open("{}_{}.pickle".format(self._model_path,trial.number), "wb") as fout:
            pickle.dump(regr, fout)

        return my_metric(regr,self._X_val, self._Y_val)
print("Done")

Creating the objective ... 
Done


In [10]:
DATA_NB = 100000
DATA_TRAIN = int(0.75*DATA_NB)
DATA_VAl = int(0.125*DATA_NB)
DATA_TEST = int(0.125*DATA_NB)

In [11]:
new_X_train = [[[[(int(val)&(1<<(i*6+j)))==(1<<(i*6+j)) for val in X_train[k].split()] for j in range(7)] for i in range(6)] for k in range(DATA_TRAIN)]
new_X_val = [[[[(int(val)&(1<<(i*6+j)))==(1<<(i*6+j)) for val in X_val[k].split()] for j in range(7)] for i in range(6)] for k in range(DATA_VAl)]
new_X_test = [[[[(int(val)&(1<<(i*6+j)))==(1<<(i*6+j)) for val in X_test[k].split()] for j in range(7)] for i in range(6)] for k in range(DATA_TEST)]

In [12]:
new_Y_train = Y_train[:DATA_TRAIN]
new_Y_val = Y_val[:DATA_VAl]
new_Y_test = Y_test[:DATA_TEST]
print(np.array(new_X_train).shape)
print(np.array(new_Y_train).shape)
print(np.array(new_X_val).shape)
print(np.array(new_Y_val).shape)
print(np.array(new_X_test).shape)
print(np.array(new_Y_test).shape)

(75000, 6, 7, 2)
(75000, 2)
(12500, 6, 7, 2)
(12500, 2)
(12500, 6, 7, 2)
(12500, 2)


In [13]:
L = np.array(new_X_train)
print(L.shape)
print(L.reshape((len(L),-1,1)).shape)
M = L.reshape((len(L),-1,1))
print(np.squeeze(M).shape)
N = np.squeeze(M)


(75000, 6, 7, 2)
(75000, 84, 1)
(75000, 84)


### Symetrie
Extend with the vertical axis symetrie

In [14]:
new_X_train += [[new_X_train[i][j][::-1] for j in range(len(new_X_train[i]))] for i in range(len(new_X_train))]
new_Y_train += new_Y_train

new_X_val += [[new_X_val[i][j][::-1] for j in range(len(new_X_val[i]))] for i in range(len(new_X_val))]
new_Y_val += new_Y_val

new_X_test += [[new_X_test[i][j][::-1] for j in range(len(new_X_test[i]))] for i in range(len(new_X_test))]
new_Y_test += new_Y_test

In [15]:
TIMEOUT = 30
NB_TRIALS = 50
STUDY_NAME = "100k_50_trials"
STR_PATH = "sqlite:///"+STUDY_NAME+"/"+STUDY_NAME+".db"

In [19]:
if os.path.exists(STUDY_NAME):
    print("-------------------------------------")
    print("Study {} loaded".format(STUDY_NAME))
    print("-------------------------------------")
    study = optuna.load_study(study_name=STUDY_NAME, storage=STR_PATH)

else:
    print("New study {} created".format(STUDY_NAME))
    os.mkdir(STUDY_NAME)
    study = optuna.create_study(study_name=STUDY_NAME,direction="minimize", storage=STR_PATH)

# Execute an optimization by using an `Objective` instance.
study.optimize(Objective(np.squeeze(np.array(new_X_train).reshape([len(new_X_train),-1,1])),  new_Y_train, np.squeeze(np.array(new_X_val).reshape([len(new_X_val),-1,1])),  new_Y_val, STUDY_NAME+"/"+STUDY_NAME), n_trials  = NB_TRIALS)

trial = study.best_trial
print("******************************")
print("******************************")
print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

-------------------------------------
Study 100k_50_trials loaded
-------------------------------------


KeyboardInterrupt: 

In [16]:
# To do, faire un vrai réseau réfléchi en utilisant optuna pour opti 

# Peut être essayer un GAN

# Utiliser symétrie

'''
New study TEST created
[I 2021-04-22 21:31:34,485] A new study created in RDB with name: TEST
81093/81093 [==============================] - 2914s 36ms/step - loss: 0.2307 - accuracy: 0.5718
[I 2021-04-22 23:45:29,873] Trial 0 finished with value: 0.2179492861032486 and parameters: {}. Best is trial 0 with value: 0.2179492861032486.
******************************
******************************
Accuracy: 0.2179492861032486
Best hyperparameters: {}'''
 '''
 [I 2021-04-23 23:45:19,568] Trial 48 finished with value: 0.0011509709795024287 and parameters: {'n_layers': 3, '0': 114, '1': 57, '2': 89, 'regularization_term': 0.5602672200939348, 'activation_function': 'tanh', 'solver': 'lbfgs', 'max_iteration': 86352764}. Best is trial 48 with value: 0.0011509709795024287.
  '''

In [16]:
STUDY_NAME = "100k_50_trials"
STR_PATH = "sqlite:///"+STUDY_NAME+"/"+STUDY_NAME+".db"
loaded_study = optuna.load_study(study_name=STUDY_NAME, storage=STR_PATH)
trial = loaded_study.best_trial
print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

Accuracy: 0.0011509709795024287
Best hyperparameters: {'0': 114, '1': 57, '2': 89, 'activation_function': 'tanh', 'max_iteration': 86352764, 'n_layers': 3, 'regularization_term': 0.5602672200939348, 'solver': 'lbfgs'}


In [17]:
print(np.squeeze(np.array(new_X_test).reshape([len(new_X_test),-1,1])).shape)
print(np.squeeze(np.array(new_X_train).reshape([len(new_X_train),-1,1])).shape)
print(np.array(new_X_train).shape)
print(np.array(new_X_test).shape)

(25000, 84)
(150000, 84)
(150000, 6, 7, 2)
(25000, 6, 7, 2)


In [18]:
with open("{}_{}.pickle".format(STUDY_NAME+"/"+STUDY_NAME,trial.number), "rb") as fin:
    best_MLPRegressor = pickle.load(fin)
Y_pred = best_MLPRegressor.predict(np.squeeze(np.array(new_X_test).reshape([len(new_X_test),-1,1])))
print(f"Test set : R2 = {r2_score(np.array(new_Y_test), Y_pred)}, NRMSE = {nrmse(np.array(new_Y_test), Y_pred)}")

Test set : R2 = -0.05110016659159866, NRMSE = 0.09886206711244899


In [92]:
print(Y_pred[:10])
print(Y_test[:10])

[[2.17598195e-01 7.87662340e-01]
 [5.79961160e-01 4.14404062e-01]
 [2.53581422e-01 7.36040793e-01]
 [6.98905851e-01 2.71273476e-01]
 [4.48238333e-01 5.27622221e-01]
 [1.46300831e-04 9.85293907e-01]
 [6.76267880e-01 3.12269875e-01]
 [4.32447534e-01 5.78726732e-01]
 [6.20575043e-01 3.65875066e-01]
 [4.51015629e-01 5.65156180e-01]]
[[1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [0.3333333333333333, 0.6666666666666666]]
